# Imports

In [62]:
# From Imports
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
from pandas import DataFrame
from datetime import datetime
from scipy.optimize import minimize
from arch import arch_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.integrate import quad


# Alias Imports
import numpy_financial as npf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pmdarima as pm

# Imports
import math
import arch
import openpyxl
import pprint
import re
import ast


print("Import Complete")

Import Complete


# Importing Bond Trading Data

In [2]:
# Importing Excel data of 2Y and 10Y Tbills
tbill2Y = pd.read_excel("Treasury_Data_V1.xlsx", "T 4.625 02 28 25 Govt TRADES")
tbill10Y = pd.read_excel("Treasury_Data_V1.xlsx", "T 3.375 05 15 33 Govt TRADES")
#note, volume is traded in increments of $1000 USD at STGT Exchange
#note, need to clean 10Y data, bids / asks don't line up

# Making Trade Time, Trade Volume, and Ask Time into lists for 2Y Bond Data
dates2Y = list(tbill2Y["Trade Time"])
asks2Y = list(tbill2Y["Ask Time"]) # Currently Not Used

print("Excel Data Imported")

Excel Data Imported


# Cleaning Bond Trading Data

In [3]:
# Cleaning Trade Time data
clean_dates2Y = []

# Going through the range and converting dates to Numpy datetime64s
for i in range(len(dates2Y)):
    date = np.datetime64(str(dates2Y[i].date()))

    # Removing all NaT values from the list
    if not np.isnat(date):

        # Adding all dates to our clean_dates2Y list
        clean_dates2Y.append(date)

    # If no date given, we make datetime max value for easy spotting
    else:
        clean_dates2Y.append(np.datetime64(datetime.max))

# Cleaning Ask Time data
clean_asks2Y = []

# Going through the range and converting dates to Numpy datetime64s
for i in range(len(asks2Y)):
    date = np.datetime64(str(asks2Y[i].date()))

    # Removing all NaT values from the list
    if not np.isnat(date):

        # Adding all dates to our clean_asks2Y list
        clean_asks2Y.append(date)

    # If no date given, we make datetime max value for easy spotting
    else:
        clean_dates2Y.append(np.datetime64(datetime.max))

print("Data cleaned")

Data cleaned


# Calculating Average Daily Volume

In [4]:
# Adding new Date object column to data
tbill2Y['Trade Time Date'] = clean_dates2Y
tbill2Y['Ask Time Date'] = clean_asks2Y

# Calculating the total volume on each specific day
day_volumes = tbill2Y.groupby(['Trade Time Date'])['Trade Volume'].sum()

# Getting all the unique dates from Trade Time Date
unique_dates = tbill2Y['Trade Time Date'].unique()

# Getting rid of the last row
minus_max_date = np.resize(unique_dates, unique_dates.size - 1)
minus_max_volume = np.resize(day_volumes, day_volumes.size - 1)

# Creating Average Hourly Trading Volumes from Daily Volumes in Dictionary
average_volume_daily = {} #key is date, value is total_volume

# Creating dictionary around unique dates, and summed daily volumes
for day, volume in zip(minus_max_date, day_volumes):

    # Calculate the average volume for the date and store it in the dictionary
    average_volume_daily[day] = volume / 12.0

# Splitting into lists of Dates, Volumes
dates = average_volume_daily.keys()
hourly_volumes = average_volume_daily.values()

# Formatting Float in Pandas
pd.options.display.float_format = '{:.2f}'.format

# Print the resulting list
daily_volumes = pd.DataFrame()
daily_volumes['Trading Day'] = dates
daily_volumes['Hourly Volume'] = hourly_volumes

print("Daily Volumes")

Daily Volumes


# Calculating Historical Midpoint between Bid and Ask

In [5]:
historical_bid_data = tbill2Y["Bid"]
historical_ask_data = tbill2Y["Ask"]
historical_midpoint = (historical_bid_data + historical_ask_data) / 2
historical_spread = abs(historical_bid_data - historical_ask_data)
# historical_midpoint

print("Calculated Historical Midpoint and Historical Spreads")

Calculated Historical Midpoint and Historical Spreads


# Calculating Price to Yield

In [6]:
def price_to_yield(price, face_value, coupon_rate, coupon_frequency, current_date, maturity_date):
    time_to_maturity_days = (maturity_date - current_date).days
    time_to_maturity_hours = time_to_maturity_days * 24  # Convert to hours

    full_coupon_periods = int(time_to_maturity_hours / (365.0 * 24 / coupon_frequency))
    partial_coupon_period_hours = time_to_maturity_hours % (365.0 * 24 / coupon_frequency)

    cash_flows = [(coupon_rate * face_value) / coupon_frequency] * full_coupon_periods

    # Handle the partial coupon period
    if partial_coupon_period_hours > 0:
        partial_coupon_payment = (coupon_rate * face_value * partial_coupon_period_hours) / (365.0 * 24)
        cash_flows.append(partial_coupon_payment)

    cash_flows[-1] += face_value  # Add the face value at maturity
    yield_value = np.nan

    try:
        yield_value = npf.irr([-price] + cash_flows)
    except ValueError:
        pass

    return yield_value

# Estimate CIR model parameters using historical yields

In [7]:
def cir_likelihood(parameters, data):
    mu, sigma, kappa, theta = parameters
    dt = 1 / (252.0 * 12)  # Hourly data assumed
    n = len(data)
    log_likelihood = 0.0

    for i in range(1, n):
        Zl = data[i - 1]
        Z = data[i]
        Z_diff = Z - Zl
        gamma = np.sqrt(kappa**2 + 2 * sigma**2)

        log_likelihood += (
            -(n - 1) * (np.log(2 * gamma) - np.log(sigma))
            - (kappa + gamma) * Z_diff
            - 2 * np.log((2 * gamma * np.exp(kappa + gamma * dt)) / (2 * gamma + (kappa + gamma) * (np.exp(gamma * dt) - 1)))
        )

    return -log_likelihood

initial_parameters = [0.05, 0.1, 0.2, 0.03]  # Initial guesses for parameters
result = minimize(cir_likelihood, initial_parameters, args=(historical_midpoint,), method='L-BFGS-B')
mu, sigma, kappa, theta = result.x
print(result.x)

[ 0.05        1.05484352 -0.00940215  0.03      ]


# Simulate bond yields based on the estimated CIR parameters

In [8]:
T = 1  # Time to maturity, years
n_simulations = 1  # Num of sims
n_periods = 252 * 12 # Num of hourly periods in a year
coupon_rate = 0.04625
coupon_frequency = 2

simulated_yields = np.zeros((n_simulations, n_periods)) #init array
bond_prices = np.zeros((n_simulations, n_periods)) #init array

for i in range(n_simulations):
    dt = T / n_periods
    current_date = tbill2Y["Bid Time"][0]
    maturity_date = current_date + pd.DateOffset(years=T)
    bond_price = historical_midpoint.iloc[0]  # Initial bond price at first midpoint value

    for j in range(1, n_periods):
        bond_yield = price_to_yield(bond_price, 1000, coupon_rate, coupon_frequency, current_date, maturity_date)
        gamma = np.sqrt(kappa**2 + 2 * sigma**2)
        bond_yield += kappa * (theta - bond_yield) * dt + sigma * np.sqrt(bond_yield) * np.random.normal(0, np.sqrt(dt))  # CIR process
        bond_price = 1000 / ((1 + bond_yield / coupon_frequency) ** (coupon_frequency * (maturity_date - current_date).days / (365 * 24)))  # Convert yield to price
        bond_prices[i, j] = bond_price
        simulated_yields[i, j] = bond_yield

cols = ["sim_bond_prices", "sim_volume", "sim_spread"]
sim_df = pd.DataFrame(columns=cols)

bond_prices_t = bond_prices.transpose()

for i in range(bond_prices_t.shape[0]):
  sim_df.loc[i,'sim_bond_prices'] = bond_prices_t[i]

print(sim_df)

          sim_bond_prices sim_volume sim_spread
0                   [0.0]        NaN        NaN
1     [959.3326186192727]        NaN        NaN
2     [998.8929243330615]        NaN        NaN
3     [999.2155137450881]        NaN        NaN
4     [999.3803074730022]        NaN        NaN
...                   ...        ...        ...
3019  [999.3098453853846]        NaN        NaN
3020  [999.5032391907814]        NaN        NaN
3021  [999.4287346842825]        NaN        NaN
3022  [999.2651058227978]        NaN        NaN
3023  [999.2656375528436]        NaN        NaN

[3024 rows x 3 columns]


# Simulate Risk Free Rate Yields

In [9]:
T = 1  # Time to maturity, years
n_simulations = 1  # Num of sims
n_periods = 252 * 12 # Num of hourly periods in a year
coupon_rate = 0.04625
coupon_frequency = 2

simulated_yields_risk_free_rate = np.zeros((n_simulations, n_periods)) #init array

for i in range(n_simulations):
    dt = T / n_periods
    current_date = tbill2Y["Bid Time"][0]
    maturity_date = current_date + pd.DateOffset(years=T)

    for j in range(1, n_periods):
        bond_yield = .0525
        gamma = np.sqrt(kappa**2 + 2 * sigma**2)
        bond_yield += kappa * (theta - bond_yield) * dt + sigma * np.sqrt(bond_yield) * np.random.normal(0, np.sqrt(dt))  # CIR process
        simulated_yields[i, j] = bond_yield

print("Simulated Yields Risk Free Rate")

Simulated Yields Risk Free Rate


# Estimating OU (Ornstein–Uhlenbeck) Process Paramters for Spread Simulation

In [10]:
# Historical spread data is used from above calculation
# Function to calculate the OU likelihood
def ou_likelihood(parameters, data):
    mean_reversion, vol, initial_spread_min = parameters
    dt = 1  # Time step (you can adjust this based on your data frequency)
    log_likelihood = 0.0
    initial_spread = initial_spread_min

    for i in range(1, len(historical_spread)):
        spread_diff = historical_spread[i] - initial_spread
        log_likelihood += -0.5 * (spread_diff / vol) ** 2
        log_likelihood -= 0.5 * np.log(2 * np.pi * vol ** 2 * dt)
        initial_spread += mean_reversion * (initial_spread_min - initial_spread) * dt

    return -log_likelihood

# Initial parameter guesses
intial_mean_reversion = 0.1
intial_vol = 0.05
intial_min_arg = historical_spread[0]

# Packing them into parameter tupe
initial_parameters = (intial_mean_reversion, intial_vol, intial_min_arg)

# Minimize the negative log-likelihood to estimate OU parameters
result = minimize(ou_likelihood, initial_parameters, args=(historical_spread,), method='L-BFGS-B')

# Extract estimated parameters
mean_reversion, volatility, initial_spread = result.x

print("OU Likelihood")

OU Likelihood


# Simulating Spreads with OU Process

In [11]:
n_simulations = 1  # Num of sims
n_periods = 252 * 12 # Num of hourly periods in a year

simulated_spreads = np.zeros((n_simulations, n_periods)) #init array

for i in range(n_simulations):
    spread = initial_spread

    for j in range(1, n_periods):
        spread += mean_reversion * (initial_spread - spread) * dt + volatility * np.sqrt(dt) * np.random.normal(0, 1)
        simulated_spreads[i, j] = spread

simulated_spreads_t = simulated_spreads.transpose()

for i in range(simulated_spreads_t.shape[0]):
     sim_df.loc[i,'sim_spread'] = simulated_spreads_t[i]

print(sim_df["sim_spread"])
print("Spreads Simulated with OU")

0                        [0.0]
1        [0.01091743144891967]
2       [0.010917154050081085]
3       [0.010953040545859296]
4       [0.011230588353507837]
                 ...          
3019    [0.012459559287275797]
3020    [0.012297979517413064]
3021    [0.012279658972909197]
3022    [0.012147384375846487]
3023    [0.012335274337811536]
Name: sim_spread, Length: 3024, dtype: object
Spreads Simulated with OU


# Simulating Volume with Normal Distribution

In [12]:
# Replace with volume data
# Calculate the average trading rate (λ) from historical data

hourly_periods = 252 * 12 # Hourly Trading Periods

# Calculate sums of Hourly Volume
total_historical_sum = 0
for hourly_volume in hourly_volumes:
    total_historical_sum += hourly_volume
average_historical_trading_rate = total_historical_sum / len(hourly_volumes)

# Simulate trading volume using a Poisson process
simulated_hourly_volume = np.random.poisson(average_historical_trading_rate, hourly_periods)

# Calculate sums of Hourly Volume
total_simulated_sum = 0
for hourly_volume in simulated_hourly_volume:
    total_simulated_sum += hourly_volume
average_simulated_trading_rate = total_simulated_sum / len(simulated_hourly_volume)

# Adjust for daily variation (for example, increase trading rate during high-activity hours)
# Can customize this adjustment based on your market's characteristics
simulated_hourly_volume[0] *= 1.2  # Increase volume during market open
simulated_hourly_volume[-1] *= 1.2  # Increase volume during market close

# Normalize simulated volume to match historical data
np.multiply(simulated_hourly_volume, total_historical_sum / total_simulated_sum)

# Create a DataFrame to store the simulated volume data
simulated_data = pd.DataFrame({'Hourly_Volume': simulated_hourly_volume})

for i in range(simulated_hourly_volume.shape[0]):
     sim_df.loc[i,'sim_volume'] = simulated_hourly_volume[i]

print(sim_df)

          sim_bond_prices sim_volume              sim_spread
0                   [0.0]   44182918                   [0.0]
1     [959.3326186192727]   36817841   [0.01091743144891967]
2     [998.8929243330615]   36809291  [0.010917154050081085]
3     [999.2155137450881]   36796100  [0.010953040545859296]
4     [999.3803074730022]   36823909  [0.011230588353507837]
...                   ...        ...                     ...
3019  [999.3098453853846]   36817511  [0.012459559287275797]
3020  [999.5032391907814]   36811599  [0.012297979517413064]
3021  [999.4287346842825]   36820855  [0.012279658972909197]
3022  [999.2651058227978]   36816533  [0.012147384375846487]
3023  [999.2656375528436]   44169519  [0.012335274337811536]

[3024 rows x 3 columns]


C:\Users\prana\AppData\Local\Temp\ipykernel_52412\3833472531.py:18: RuntimeWarning: overflow encountered in scalar add
  total_simulated_sum += hourly_volume


# Predicted Loss Functions



In [76]:
def alpha_t(k, Z, spread):
    Zu = Z + spread / 2
    Zl = Z - spread / 2
    alpha = 0
    if Zl < Z < Zu:
        xi = k * (Z**(1/2) - (Zl)**(1/2))
        yi = k * (Z**(-1/2) - Zu**(-1/2))
        alpha = xi + yi * Z
    elif Z < Zl:
        xi = 0
        yi = k * ((Zl)**(-1/2) - (Zu)**(-1/2))
        alpha = xi + yi * Z
    elif Z > Zu:
        xi = k * ((Zu)**(1/2) - (Zl)**(1/2))
        yi = 0
        alpha = xi + yi * Z
    return alpha

#See denominator or convexity cost
def LP_spread_conversion(Z, spread):
    Zu = Z + spread / 2
    Zl = Z - spread / 2
    sigma_u = -2 * (Z**(1/2)) / (Zu**(1/2)) + 2
    sigma_l = -2 * (Z**(1/2)) / (Zl**(1/2)) + 2
    return sigma_u + sigma_l


#only intake spread, convert to Zl Zu

#k is the value of x * y, in this case we know volume, x is USDC and y is bond contracts, y is given by volume, quantity of usdc is price of contract * number contracts traded
#k = (contract_volume**2) * price


def calculate_convexity_cost(volume_values, spread_values, Z_values):
    # Initialize an empty list to store the results
    convexity_cost = []

    for i in range(len(Z_values)):
        dZZ = 0 if i == 0 else (Z_values[i] - Z_values[i - 1])**2
        converted_spreads = LP_spread_conversion(Z_values[i], spread_values[i])
        alpha_lu = volume_values[i] + volume_values[i] * Z_values[i]
        price_variation = dZZ  # Use the quadratic variation here
        
        # Calculate the component of the convexity cost
        component = alpha_lu / (2 * converted_spreads * (Z_values[i])**2) * price_variation
        convexity_cost.append(component)

    # Calculate the integral
    integral = np.trapz(convexity_cost)
    
    return integral


def calculate_opportunity_cost(alpha_0, alpha_d, risk_free_rate_data, dt):
    opportunity_cost = 0.0
    for i in range(len(alpha_d)):
        opportunity_cost += (alpha_d[i] - alpha_0) * risk_free_rate_data[i] * dt
    return opportunity_cost

price_forecasts = pd.read_csv("price_forecasts_60_steps.csv")
spread_forecasts = pd.read_csv("spread_forecasts_60_steps.csv")
volume_forecasts = pd.read_csv("volumes_forecasts_60_steps.csv")



In [24]:
#ANISH CALCULATED RESULTS

price_forecasts_list = price_forecasts.values.tolist()
spread_forecasts_list = spread_forecasts.values.tolist()
volume_forecasts_list = volume_forecasts.values.tolist()

integral = calculate_conv_cost(volume_forecasts_list, spread_forecasts_list, price_forecasts_list)
new_sim_df = sim_df.head(2765)
opportunity_costs = []

for t in range(len(price_forecasts_list)):
    
    alpha_0 = alpha_t(new_sim_df['sim_bond_prices'][t] * new_sim_df['sim_volume'][t]** 2, new_sim_df['sim_bond_prices'][t], new_sim_df["sim_spread"][t])
    
    opportunity_cost = 0
    
    for step in range(60):
        forecasted_z = float(re.sub(r"[^\d.-]", "", price_forecasts_list[t][step]))
        forecasted_vol = volume_forecasts_list[t][step]
        forecasted_spread = float(re.sub(r"[^\d.-]", "", spread_forecasts_list[t][step]))

        alpha_d = alpha_t(forecasted_z * forecasted_vol ** 2, forecasted_z, forecasted_spread)
        opportunity_cost += calculate_opportunity_cost(alpha_0, alpha_d, simulated_yields, 1)

    opportunity_costs.append(opportunity_cost)


oc_flat_list = np.array(opportunity_costs).flatten()
total_oc = sum(oc_flat_list)
print("TOTAL OC: ",total_oc)

2765
8361360


In [107]:
#PRANAV REORG STARTS HERE
new_sim_df = sim_df.head(2765)


#simulated_yields = simulated_yields.tolist()
rfr_rates = simulated_yields[0]
rfr_rates = rfr_rates[:2765]


#Calculating both convexity and opportunity cost
    
    
aggregated_volume_forecast = []
total_forecasted_PL = []


for i in range(len(new_sim_df)):
    period_price = float(new_sim_df["sim_bond_prices"][i])
    period_volume = float(new_sim_df["sim_volume"][i])
    period_spread = float(new_sim_df["sim_spread"][i])
    
    price_forecasts_60 = price_forecasts.iloc[i].to_list()
    price_forecasts_60 = [float(item.strip("[]'")) for item in price_forecasts_60]
    
    spread_forecasts_60 = spread_forecasts.iloc[i].to_list()
    spread_forecasts_60 = [float(item.strip("[]'")) for item in spread_forecasts_60]
    spread_forecasts_60 = spread_forecasts_60 * 10 #price on 1000 scale
    
    volumes_forecasts_60 = volume_forecasts.iloc[i].to_list()
    volumes_forecasts_60 = [float(item) for item in volumes_forecasts_60]
    #k_forecasts_60 = [volumes_forecasts_60[i]**2 * price_forecasts_60[i] for i in range(len(volumes_forecasts_60))]  
    aggregated_volume_forecast.append(sum(volumes_forecasts_60))
    
    alpha_0 = alpha_t(period_volume, period_price, period_spread)
    alpha_d = [alpha_t(volumes_forecasts_60[j], price_forecasts_60[j], spread_forecasts_60[j]) for j in range(len(price_forecasts_60))]
    
    forecasted_convexity_cost = calculate_convexity_cost(volumes_forecasts_60, spread_forecasts_60, price_forecasts_60)
    forecasted_opportunity_cost = calculate_opportunity_cost(alpha_0, alpha_d, rfr_rates, 1)
    
    total_forecasted_PL.append((forecasted_convexity_cost + forecasted_opportunity_cost) * -1)
    


In [106]:
#Predicted loss has been calculated, what metrics are left?
#fee policy, fee_earned_AMM, realized_PL, fees_aggregated, realized PnL

sim_bond_prices = new_sim_df["sim_bond_prices"].to_list()
sim_spreads = new_sim_df["sim_spread"].to_list()
sim_volumes = new_sim_df["sim_volume"].to_list()

#Calculating fee policy
fee_policy = [total_forecasted_PL[i] / aggregated_volume_forecast[i] for i in range(len(total_forecasted_PL))]
fee_policy


#Determining whether fee was earned based on prev period LP position
fee_earned_AMM = []
for i in range(len(sim_df)):
    price_prev = sim_bond_prices[i-1]
    upper_bound_prev = price_prev + sim_spreads[i-1] / 2
    lower_bound_prev = price_prev - sim_spreads[i-1] / 2
    
    if i == 0: 
        fee = 0
    elif lower_bound_prev < sim_bond_prices[i] < upper_bound_prev:
        fee = sim_volumes[i-1] * fee_policy[i-1]
    else:
        fee = 0
    fee_earned_AMM.append(fee)
    


fees_aggregated = [sum(fee_earned_AMM[:i]) for i in range(len(new_sim_df))]


fees_aggregated

'''

 
fees_aggregated --> aggregate above

realized_PL --> calculate PL for all periods before present (uses sim_df values, NOT forecasts)


realized_PnL = fees_aggregated - realized_PL


trad_exchange_profit --> spread * volume, with same logic as fee_earned_AMM


1. plot fee policy over time
2. plot PnL over time
3. plot profit of AMM vs trad_exchange_profits


'''


IndexError: list index out of range

In [103]:
''''

,sim_bond_prices,sim_volume,sim_spread
0,[0.0],44182918,[0.0]
1,[959.3326186192727],36817841,[0.01091743144891967]
2,[998.8929243330615],36809291,[0.010917154050081085]
3,[999.2155137450881],36796100,[0.010953040545859296]
4,[999.3803074730022],36823909,[0.011230588353507837]
...,...,...,...
3019,[999.3098453853846],36817511,[0.012459559287275797]
3020,[999.5032391907814],36811599,[0.012297979517413064]
3021,[999.4287346842825],36820855,[0.012279658972909197]
3022,[999.2651058227978],36816533,[0.012147384375846487]


# Calculating Fee Policy Using Forecasting

In [ ]:
# Define training and testing sizes
training_size = 200
testing_size = 6
Z = bond_prices.T



# Initialize the ARIMA-GARCH model
arima_model = pm.auto_arima(Z[:training_size], suppress_warnings=True)
garch_model = arch.arch_model(Z[:training_size], vol='Garch', p=1, q=1)

# Lists to store performance metrics and forecasts
mae_list = []
mse_list = []
forecasts = []

# Function to convert log price differences to prices
def log_diffs_to_prices(log_diffs, initial_price):
    prices = [initial_price]
    for log_diff in log_diffs:
        price = prices[-1] * math.exp(log_diff)
        prices.append(price)
    return prices[1:]  # Remove the initial price

log_diffs = []

for i in range(1, len(Z)):
    previous_price = Z[i - 1]
    current_price = Z[i]

    # Check if both prices are positive before calculating log difference
    if previous_price > 0 and current_price > 0:
        log_diff = math.log(current_price) - math.log(previous_price)
        log_diffs.append(log_diff)
    else:
        # Handle the case where either price is not positive (e.g., set log_diff to 0)
        log_diffs.append(0.0)


# Iterate through the data using ROCV
for i in range(training_size, len(Z) - testing_size + 1):
    training_data = log_diffs[i - training_size:i]
    testing_data = log_diffs[i:i + testing_size]

    # Re-estimate the ARIMA model with new training data
    arima_model.fit(training_data)
    best_arima_order = arima_model.order

    # Re-estimate the GARCH model with new training data
    garch_model = arch.arch_model(training_data, vol='Garch', p=1, q=1)
    res = garch_model.fit(disp='off')

    # Forecast the next value using the updated model
    forecast = arima_model.predict(n_periods=testing_size)
    volatility = res.conditional_volatility[-1]
    forecast = forecast * np.sqrt(volatility)

    # Convert log differences back to prices using the most recent price in the training data
    initial_price = Z[i - 1]
    forecast_prices = log_diffs_to_prices(forecast, initial_price)

    # Store the 6-step ahead forecasts
    forecasts.append(forecast_prices)

    # Calculate evaluation metrics
    mae = mean_absolute_error(testing_data, forecast)
    mse = mean_squared_error(testing_data, forecast)

    mae_list.append(mae)
    mse_list.append(mse)

print(forecasts.shape())

/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 7.975e-06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:765: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 7.975e-06. Parameter
estimation work better when this value is between 1 and 1000. The recomm

KeyboardInterrupt: ignored

In [ ]:
forecasts

# Create a DataFrame from the forecasts data
df = pd.DataFrame(forecasts, columns=[f'Step_{i+1}' for i in range(len(forecasts[0]))])

# Specify the file path where you want to save the data
file_path = 'price_forecasts.csv'  # Change the file extension to .xlsx for Excel

# Save the DataFrame to a CSV or Excel file
df.to_csv(file_path, index=False)  # For CSV
# df.to_excel(file_path, index=False)  # For Excel

print(f'Forecasts data saved to {file_path}')

Forecasts data saved to forecasts.csv


In [ ]:

#6 steps ahead volume forecast

#Calculate expected predicted loss at each time period based on the 6 steps ahead forecast data

#At each point Sum forecasted volume for the next 6 periods together, call this forecasted_total_volume, save dynamic_fee_percent = aggregate_forecasted_PL /  forecasted_total_volume

#Realized_fee_profit = prev period "actual" volume multiplied by prev period fee_percent, IFF the current price is within the range of LP position in the last period

#Chart fee_percent and realized_fee_profit



In [ ]:
#Volume forecasts

spreads = sim_df["sim_spread"].to_list()

training_size = 200
testing_size = 6
Z = spreads



# Initialize the ARIMA-GARCH model
arima_model = pm.auto_arima(Z[:training_size], suppress_warnings=True)
garch_model = arch.arch_model(Z[:training_size], vol='Garch', p=1, q=1)

# Lists to store performance metrics and forecasts
mae_list = []
mse_list = []
forecasts = []

# Function to convert log price differences to prices
def log_diffs_to_prices(log_diffs, initial_price):
    prices = [initial_price]
    for log_diff in log_diffs:
        price = prices[-1] * math.exp(log_diff)
        prices.append(price)
    return prices[1:]  # Remove the initial price

log_diffs = []

for i in range(1, len(Z)):
    previous_price = Z[i - 1]
    current_price = Z[i]

    # Check if both prices are positive before calculating log difference
    if previous_price > 0 and current_price > 0:
        log_diff = math.log(current_price) - math.log(previous_price)
        log_diffs.append(log_diff)
    else:
        # Handle the case where either price is not positive (e.g., set log_diff to 0)
        log_diffs.append(0.0)




# Iterate through the data using ROCV
for i in range(training_size, len(Z) - testing_size + 1):
    training_data = log_diffs[i - training_size:i]
    testing_data = log_diffs[i:i + testing_size]

    # Re-estimate the ARIMA model with new training data
    arima_model.fit(training_data)
    best_arima_order = arima_model.order

    # Re-estimate the GARCH model with new training data
    garch_model = arch.arch_model(training_data, vol='Garch', p=1, q=1)
    res = garch_model.fit(disp='off')

    # Forecast the next value using the updated model
    forecast = arima_model.predict(n_periods=testing_size)
    volatility = res.conditional_volatility[-1]
    forecast = forecast * np.sqrt(volatility)

    # Convert log differences back to prices using the most recent price in the training data
    initial_price = Z[i - 1]
    forecast_prices = log_diffs_to_prices(forecast, initial_price)

    # Store the 6-step ahead forecasts
    forecasts.append(forecast_prices)

    # Calculate evaluation metrics
    #mae = mean_absolute_error(testing_data, forecast)
    #mse = mean_squared_error(testing_data, forecast)

    #mae_list.append(mae)
    #mse_list.append(mse)

print("Done")

C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 6.676e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 6.829e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: Data

C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 6.929e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 6.964e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: Data

C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 7.741e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 7.729e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: Data

C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 8.55e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 8.76e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataSc

C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 9.216e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 9.15e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataS

...

C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 5.593e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 5.588e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: Data

C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 5.527e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 5.517e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: Data

C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 5.567e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 5.6e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataSc

Done


C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 5.326e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 5.348e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\prana\anaconda3\Lib\site-packages\arch\univariate\base.py:311: Data

In [ ]:
forecasts

# Create a DataFrame from the forecasts data
df = pd.DataFrame(forecasts, columns=[f'Step_{i+1}' for i in range(len(forecasts[0]))])

# Specify the file path where you want to save the data
file_path = 'spread_forecasts.csv'  # Change the file extension to .xlsx for Excel

# Save the DataFrame to a CSV or Excel file
df.to_csv(file_path, index=False)  # For CSV
# df.to_excel(file_path, index=False)  # For Excel

print(f'Forecasts data saved to {file_path}')

Forecasts data saved to spread_forecasts.csv
